# IST597:- Multi-Layer Perceptron

## Load the libraries

In [7]:
import os
import numpy as np
import time
import tensorflow as tf
seed = 5308
np.random.seed(seed)
tf.random.set_seed(seed)

In [8]:
tf.config.list_physical_devices('GPU')

[]

In [9]:
tf.executing_eagerly()

True

#Get number of Gpu's and id's in the system or else you can also use Nvidia-smi in command prompt.

## Generate random data

In [10]:
#Getting the datasets - mnist : digit dataset, fashion_mnist : labelled fashion images
mnist = tf.keras.datasets.mnist
#mnist = tf.keras.datasets.fashion_mnist

In [12]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, X_test.shape

11501568/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (10000, 28, 28))

In [22]:
size_input = 784
size_hidden_1 = 100 
size_hidden_2 = 100
size_output = 10 #should be 10 since digits are from 0-9
number_of_train_examples = X_train.shape[0]
number_of_test_examples = X_test.shape[0]
print (f"{number_of_test_examples},{number_of_train_examples}")

10000,60000


In [ ]:
#Normalizing data
#X_train = X_train/255.0
#X_test = X_test/255.0

In [16]:
X_train = X_train.reshape(number_of_train_examples, -1)
X_test = X_test.reshape(number_of_test_examples, -1)
X_test.shape, X_train.shape

((10000, 784), (60000, 784))

In [17]:
#Normalizing data
X_train = X_train/255.0
X_test = X_test/255.0

In [18]:
y_train[0]

5

In [19]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [20]:
# Split dataset into batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)
train_ds,test_ds

(<BatchDataset shapes: ((None, 784), (None, 10)), types: (tf.float64, tf.float32)>,
 <BatchDataset shapes: ((None, 784), (None, 10)), types: (tf.float64, tf.float32)>)

# Build MLP using Eager Execution

## Combined model with L2 and dropout enable knobs

In [23]:
# Combined model with L2 and dropout enable knobs
import sys
from tensorflow.python.ops.gen_math_ops import square
import numpy as np
# Define class to build mlp model
class MLP_combined(object):
  def __init__(self, size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "default", lambda_l2 = 0.8, dropout = 0.3, lr = 0.05, device=None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    import sys
    self.size_input, self.size_hidden_1, self.size_hidden_2, self.size_output, self.reg_mode, self.lambda_l2, self.dropout, self.lr, self.device =\
    size_input, size_hidden_1, size_hidden_2, size_output, reg_mode, lambda_l2, dropout, lr, device
    
    # Initialize weights between input layer and hidden layer 1 - (128, 128)
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden_1]))
    # Initialize biases for hidden layer 1 - (1,128)
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden_1]))

    # Initialize weights between hidden layer 1 and hidden layer 2 - (128, 128)
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden_1, self.size_hidden_2]))
    # Initialize biases for hidden layer 2 - (1, 128)
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden_2]))

    # Initialize weights between hidden layer 2 and output layer - (128, 10)
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden_2, self.size_output]))
    # Initialize biases for output layer - (1,10)
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))

    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3]
    
  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y
  
  def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''

    #Use categorical cross entropy in place of MSE
    cce = tf.keras.losses.CategoricalCrossentropy()
    #loss = tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred)
    loss = cce(y_true, y_pred)

    if self.reg_mode == "L2":
      #Applying L2 regularization
      #print ("#Applying L2 regularization")
      para_list = [self.W1, self.W2, self.W3]
      regularizer = sum([tf.reduce_sum(tf.square(_var)) for _var in para_list])
      #loss = self.lambda_l2 * loss +  ((1 - self.lambda_l2)/ number_of_train_examples) * regularizer
      loss = loss +  ((self.lambda_l2/(2 * number_of_train_examples)) * regularizer)

    return loss
  
  def backward(self, X_train, y_train):
    """
    backward pass
    """
    optimizer = tf.keras.optimizers.SGD(learning_rate=self.lr)
    #optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
      grads = tape.gradient(current_loss, self.variables)
      optimizer.apply_gradients(zip(grads, self.variables))
        
        
  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    
    #Remember to normalize your dataset before moving forward
    
    # Compute values in hidden layer 1
    w1hat = tf.matmul(X_tf, self.W1) + self.b1
    h1hat = tf.nn.relu(w1hat)

    # Compute values in hidden layer 2
    w2hat = tf.matmul(h1hat, self.W2) + self.b2
    h2hat = tf.nn.relu(w2hat)

    if self.reg_mode == "dropout":
      # Adding dropout on layer 2
      #print ("# Adding dropout on layer 2")
      h2hat = tf.nn.dropout(h2hat, self.dropout)

    # Compute output
    output = tf.matmul(h2hat, self.W3) + self.b3
    #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
    #Second add tf.Softmax(output) and then return this variable
    output = tf.nn.softmax(output)
    #print (f"Output : {output}")
    return output

## Train Model

# Function to train a model and prints cross entropy and accuracy

In [24]:
# Function to train a model and prints cross entropy and accuracy
from tqdm import tqdm
def train_model(mlp, num_epochs, seed, shuffle_size, batch_size):
  time_start = time.time()
  for epoch in tqdm(range(0, num_epochs)):
    loss_total = tf.zeros([1,1], dtype=tf.float32)
    correct_preds_total = 0
    #lt = 0
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(shuffle_size, seed=epoch*(seed)).batch(batch_size)
    for inputs, outputs in train_ds:
      preds = mlp.forward(inputs)
      loss_batch = mlp.loss(preds, outputs)
      loss_total = loss_total + loss_batch
      #lt = lt + mlp.loss(preds, outputs)
      
      y_pred_argmax = np.argmax(preds, 1)

      y_true_argmax = np.argmax(outputs, 1)
      
      k = (y_pred_argmax == y_true_argmax)
      correct_preds_batch = len(k[k == True])
      correct_preds_total += correct_preds_batch

      mlp.backward(inputs, outputs)
    #print (correct_preds_total)
    #sys.exit()
    accuracy_epoch = correct_preds_total / X_train.shape[0]
    print('Number of Epoch = {} - Average CCE:= {} - Average Accuracy:= {}%'.format(epoch + 1, np.sum(loss_total) / X_train.shape[0], accuracy_epoch * 100))
    
  time_taken = time.time() - time_start
  print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

In [25]:
# Set number of epochs
NUM_EPOCHS = 10
NUM_EPOCHS = 20

In [26]:
#default training
mlp_default = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "default", lr = 0.05, device='gpu')
train_model(mlp_default, NUM_EPOCHS, seed, 100,64)

  5%|▌         | 1/20 [00:20<06:37, 20.93s/it]

Number of Epoch = 1 - Average CCE:= 0.18373401692708333 - Average Accuracy:= 26.956666666666667%


 10%|█         | 2/20 [00:41<06:13, 20.72s/it]

Number of Epoch = 2 - Average CCE:= 0.16789650065104167 - Average Accuracy:= 33.31666666666667%


 15%|█▌        | 3/20 [00:56<05:04, 17.91s/it]

...

Number of Epoch = 18 - Average CCE:= 0.06466699625651041 - Average Accuracy:= 74.28666666666666%


 95%|█████████▌| 19/20 [05:29<00:15, 15.60s/it]

Number of Epoch = 19 - Average CCE:= 0.06310406901041667 - Average Accuracy:= 74.91499999999999%


100%|██████████| 20/20 [05:44<00:00, 17.20s/it]

Number of Epoch = 20 - Average CCE:= 0.06307694498697916 - Average Accuracy:= 74.91666666666667%

Total time taken (in seconds): 344.09


In [30]:
#default training
mlp_default = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "default", lr = 0.05, device='gpu')
train_model(mlp_default, NUM_EPOCHS, seed, 1000,512)

  5%|▌         | 1/20 [00:05<01:40,  5.31s/it]

Number of Epoch = 1 - Average CCE:= 0.021645084635416666 - Average Accuracy:= 31.128333333333337%


 10%|█         | 2/20 [00:08<01:15,  4.22s/it]

Number of Epoch = 2 - Average CCE:= 0.017260528564453124 - Average Accuracy:= 45.123333333333335%


 15%|█▌        | 3/20 [00:12<01:05,  3.83s/it]

...

Number of Epoch = 18 - Average CCE:= 0.00885420633951823 - Average Accuracy:= 71.78166666666667%


 95%|█████████▌| 19/20 [01:21<00:04,  4.01s/it]

Number of Epoch = 19 - Average CCE:= 0.008766328938802083 - Average Accuracy:= 72.11999999999999%


100%|██████████| 20/20 [01:26<00:00,  4.35s/it]

Number of Epoch = 20 - Average CCE:= 0.008750503540039062 - Average Accuracy:= 72.10499999999999%

Total time taken (in seconds): 86.94


In [31]:
#l2reg training
mlp_l2 = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "L2", lambda_l2 = 0.05, device='gpu')
train_model(mlp_l2, NUM_EPOCHS, seed, 1000,512)

  5%|▌         | 1/20 [00:03<01:12,  3.84s/it]

Number of Epoch = 1 - Average CCE:= 0.02552958984375 - Average Accuracy:= 19.311666666666667%


 10%|█         | 2/20 [00:07<01:09,  3.84s/it]

Number of Epoch = 2 - Average CCE:= 0.02099154459635417 - Average Accuracy:= 33.70666666666667%


 15%|█▌        | 3/20 [00:12<01:15,  4.47s/it]

...

Number of Epoch = 18 - Average CCE:= 0.008254427083333333 - Average Accuracy:= 74.01166666666667%


 95%|█████████▌| 19/20 [01:22<00:03,  3.95s/it]

Number of Epoch = 19 - Average CCE:= 0.008195360310872396 - Average Accuracy:= 74.13833333333332%


100%|██████████| 20/20 [01:26<00:00,  4.33s/it]

Number of Epoch = 20 - Average CCE:= 0.008120443725585938 - Average Accuracy:= 74.41666666666666%

Total time taken (in seconds): 86.60


In [41]:
#l2reg training
mlp_l2 = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "L2", lambda_l2 = 0.05, device='gpu')
train_model(mlp_l2, NUM_EPOCHS, seed, 100,64)

  5%|▌         | 1/20 [00:21<06:46, 21.39s/it]

Number of Epoch = 1 - Average CCE:= 0.17843375651041668 - Average Accuracy:= 29.30333333333333%


 10%|█         | 2/20 [00:38<05:41, 18.95s/it]

Number of Epoch = 2 - Average CCE:= 0.160369189453125 - Average Accuracy:= 36.525%


 15%|█▌        | 3/20 [00:59<05:34, 19.69s/it]

...

Number of Epoch = 18 - Average CCE:= 0.07251875813802083 - Average Accuracy:= 71.35166666666667%


 95%|█████████▌| 19/20 [06:03<00:19, 19.85s/it]

Number of Epoch = 19 - Average CCE:= 0.070668408203125 - Average Accuracy:= 72.1%


100%|██████████| 20/20 [06:21<00:00, 19.08s/it]

Number of Epoch = 20 - Average CCE:= 0.07031485188802084 - Average Accuracy:= 72.21833333333333%

Total time taken (in seconds): 381.63


In [36]:
#dropout training
mlp_dropout = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "dropout", dropout = 0.3, device='gpu')
train_model(mlp_dropout, NUM_EPOCHS, seed, 1000,512)

  5%|▌         | 1/20 [00:10<03:16, 10.37s/it]

Number of Epoch = 1 - Average CCE:= 0.027097235107421874 - Average Accuracy:= 14.215%


 10%|█         | 2/20 [00:15<02:12,  7.37s/it]

Number of Epoch = 2 - Average CCE:= 0.023969694010416667 - Average Accuracy:= 24.10333333333333%


 15%|█▌        | 3/20 [00:20<01:48,  6.38s/it]

...

Number of Epoch = 18 - Average CCE:= 0.013035978190104167 - Average Accuracy:= 58.656666666666666%


 95%|█████████▌| 19/20 [01:39<00:04,  4.73s/it]

Number of Epoch = 19 - Average CCE:= 0.012164452107747395 - Average Accuracy:= 61.27166666666667%


100%|██████████| 20/20 [01:44<00:00,  5.23s/it]

Number of Epoch = 20 - Average CCE:= 0.011132398478190104 - Average Accuracy:= 64.56833333333334%

Total time taken (in seconds): 104.67


In [42]:
#dropout training
mlp_dropout = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "dropout", dropout = 0.3, device='gpu')
train_model(mlp_dropout, NUM_EPOCHS, seed, 100,64)

  5%|▌         | 1/20 [00:19<06:08, 19.40s/it]

Number of Epoch = 1 - Average CCE:= 0.18236220703125 - Average Accuracy:= 27.441666666666663%


 10%|█         | 2/20 [00:39<06:01, 20.10s/it]

Number of Epoch = 2 - Average CCE:= 0.142130908203125 - Average Accuracy:= 43.45%


 15%|█▌        | 3/20 [00:58<05:31, 19.50s/it]

...

Number of Epoch = 18 - Average CCE:= 0.04922142333984375 - Average Accuracy:= 80.40333333333334%


 95%|█████████▌| 19/20 [06:13<00:18, 18.58s/it]

Number of Epoch = 19 - Average CCE:= 0.04744493408203125 - Average Accuracy:= 81.08%


100%|██████████| 20/20 [06:30<00:00, 19.51s/it]

Number of Epoch = 20 - Average CCE:= 0.047373067220052086 - Average Accuracy:= 81.12166666666667%

Total time taken (in seconds): 390.16


In [44]:
#dropout training
NUM_EPOCHS = 50
mlp_dropout = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "dropout", dropout = 0.3, device='gpu')
train_model(mlp_dropout, NUM_EPOCHS, seed, 1000,512)

  2%|▏         | 1/50 [00:05<04:17,  5.25s/it]

Number of Epoch = 1 - Average CCE:= 0.024509676106770833 - Average Accuracy:= 22.226666666666667%


  4%|▍         | 2/50 [00:09<03:34,  4.47s/it]

Number of Epoch = 2 - Average CCE:= 0.019988043212890626 - Average Accuracy:= 36.48833333333334%


  6%|▌         | 3/50 [00:14<03:46,  4.81s/it]

...

Number of Epoch = 48 - Average CCE:= 0.006144720967610677 - Average Accuracy:= 80.37666666666667%


 98%|█████████▊| 49/50 [03:49<00:04,  4.37s/it]

Number of Epoch = 49 - Average CCE:= 0.006043127950032552 - Average Accuracy:= 80.69%


100%|██████████| 50/50 [03:53<00:00,  4.67s/it]

Number of Epoch = 50 - Average CCE:= 0.00597988993326823 - Average Accuracy:= 80.88666666666666%

Total time taken (in seconds): 233.67


## One Step Inference

#function to test the model and return CCE and Accuracy

In [45]:
#function to test the model and return CCE and Accuracy
def test_model(mlp):
  test_loss_total = tf.Variable(0, dtype=tf.float32)
  correct_preds_total = 0
  #test_loss_total = 0.0
  for inputs, outputs in test_ds:
    preds = mlp.forward(inputs)
    test_loss_total = test_loss_total + mlp.loss(preds, outputs)
    y_pred_argmax = np.argmax(preds, 1)
    y_true_argmax = np.argmax(outputs, 1)
    k = (y_pred_argmax == y_true_argmax)
    correct_preds_batch = len(k[k == True])

    correct_preds_total += correct_preds_batch
  
  tot_cce = np.sum(test_loss_total.numpy()) / X_test.shape[0]
  tot_acc = correct_preds_total/X_test.shape[0]
  print('Test CCE: {:.4f}, Accuracy : {}'.format(tot_cce, tot_acc))
  return (tot_cce, tot_acc)

In [46]:
#testing default
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
test_model(mlp_default)

Test CCE: 0.0432, Accuracy : 0.7294


(0.04322297668457031, 0.7294)

In [47]:
#testing L2 model
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
test_model(mlp_l2)

Test CCE: 0.0419, Accuracy : 0.7412


(0.041940997314453124, 0.7412)

In [48]:
#testing dropout model
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
test_model(mlp_dropout)

Test CCE: 0.0306, Accuracy : 0.8077


(0.030606463623046875, 0.8077)

In [43]:
#testing dropout model
#training batch size 64 model
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
test_model(mlp_dropout)

Test CCE: 0.0293, Accuracy : 0.8173


(0.029330807495117187, 0.8173)

In [50]:
# Obtaining inferences for all three models - 10 iteration per model
from tqdm import tqdm

############## Setting values as per dataset ##############
size_input = 784
size_hidden_1 = 100 
size_hidden_2 = 100
size_output = 10 
number_of_train_examples = X_train.shape[0]
number_of_test_examples = X_test.shape[0]
print (f"{number_of_test_examples},{number_of_train_examples}")

########################################################
'''
Additional steps done above
1. Reshape to flatten
2. Normalization
3. One hot encoding
'''

#######################################################
reg_type_list = ["default", "L2", "dropout"]
inference_stats_dict = {"default":{}, "L2": {}, "dropout": {}}
lambda_val = 0.05
dropout_val = 0.3
NUM_EPOCHS = 20
random_seeds = list(np.random.randint(low=2000, high=4000, size=10))
inf_iters = 10
device_type = 'gpu'
shuffle_size = 1000
batch_size = 512
learning_rate = 0.05
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
####################################################

for reg_type in reg_type_list:
  print (f"## START: Current Model: {reg_type} ##")
  cce_list = []
  acc_list = []
  for iter in tqdm(range(inf_iters)):
    print(f"\n** Inference Iteration: {iter} **")
    cur_seed = random_seeds[iter]
    print (f"\n#Training {reg_type} model with seed {cur_seed}#")
    np.random.seed(cur_seed)
    tf.random.set_seed(cur_seed)

    # Model initialization as per regularization type - regularization code triggers only if knob is on
    mlp = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = reg_type, lambda_l2 = lambda_val, dropout = dropout_val, lr = learning_rate, device=device_type)
    train_model(mlp, NUM_EPOCHS, cur_seed, shuffle_size, batch_size)

    # Testing the data
    (cce, acc) = test_model(mlp)
    print(f"seed: {cur_seed}, Cross Entropy loss: {cce}, Accuracy: {acc}")

    cce_list.append(cce)
    acc_list.append(acc)

  inference_stats_dict[reg_type]["cce_list"] = cce_list
  inference_stats_dict[reg_type]["acc_list"] = acc_list
  print (f"## END: Current Model: {reg_type}##")
  print (f"Current inference results: {inference_stats_dict}")
################################   

10000,60000
## START: Current Model: default ##


  0%|          | 0/10 [00:00<?, ?it/s]


** Inference Iteration: 0 **

#Training default model with seed 3058#



  5%|▌         | 1/20 [00:06<01:59,  6.28s/it]

Number of Epoch = 1 - Average CCE:= 0.025529276529947915 - Average Accuracy:= 19.183333333333334%


...

Number of Epoch = 19 - Average CCE:= 0.007450501505533854 - Average Accuracy:= 76.13666666666667%



100%|██████████| 20/20 [01:34<00:00,  4.70s/it]

Number of Epoch = 20 - Average CCE:= 0.00727490234375 - Average Accuracy:= 76.735%

Total time taken (in seconds): 94.02



100%|██████████| 10/10 [15:55<00:00, 95.56s/it]

Test CCE: 0.0367, Accuracy : 0.7694
seed: 3155, Cross Entropy loss: 0.03669295654296875, Accuracy: 0.7694
## END: Current Model: dropout##
Current inference results: {'default': {'cce_list': [0.0748375244140625, 0.05811812744140625, 0.08847588500976562, 0.029572921752929686, 0.061538922119140624, 0.08598414916992188, 0.07229287719726563, 0.07354020385742188, 0.08425131225585937, 0.07390631713867188], 'acc_list': [0.5342, 0.6372, 0.4499, 0.8153, 0.616, 0.465, 0.5498, 0.5418, 0.4765, 0.5408]}, 'L2': {'cce_list': [0.05077794799804688, 0.056246685791015626, 0.09849248657226563, 0.05702962646484375, 0.06205078125, 0.09080116577148438, 0.07283306884765625, 0.07203131103515625, 0.0712072998046875, 0.07413899536132812], 'acc_list': [0.6847, 0.6514, 0.3908, 0.647, 0.6157, 0.4369, 0.5493, 0.5535, 0.5597, 0.5414]}, 'dropout': {'cce_list': [0.026508352661132814, 0.05739667358398438, 0.05832355346679687, 0.039481912231445315, 0.05254434814453125, 0.06762960205078125, 0.03944203796386719, 0.03917510

In [51]:
print ("###### Final stats ##########")
print (inference_stats_dict)

###### Final stats ##########
{'default': {'cce_list': [0.0748375244140625, 0.05811812744140625, 0.08847588500976562, 0.029572921752929686, 0.061538922119140624, 0.08598414916992188, 0.07229287719726563, 0.07354020385742188, 0.08425131225585937, 0.07390631713867188], 'acc_list': [0.5342, 0.6372, 0.4499, 0.8153, 0.616, 0.465, 0.5498, 0.5418, 0.4765, 0.5408]}, 'L2': {'cce_list': [0.05077794799804688, 0.056246685791015626, 0.09849248657226563, 0.05702962646484375, 0.06205078125, 0.09080116577148438, 0.07283306884765625, 0.07203131103515625, 0.0712072998046875, 0.07413899536132812], 'acc_list': [0.6847, 0.6514, 0.3908, 0.647, 0.6157, 0.4369, 0.5493, 0.5535, 0.5597, 0.5414]}, 'dropout': {'cce_list': [0.026508352661132814, 0.05739667358398438, 0.05832355346679687, 0.039481912231445315, 0.05254434814453125, 0.06762960205078125, 0.03944203796386719, 0.03917510681152344, 0.06364664306640624, 0.03669295654296875], 'acc_list': [0.8314, 0.6419, 0.6363, 0.7524, 0.6714, 0.5779, 0.7522, 0.7548, 0.603

In [52]:
import pandas as pd

for ele in inference_stats_dict:
  print (f"## Model: {ele} ###")
  print (f"Cross entropy loss across trials : {inference_stats_dict[ele]['cce_list']}")
  print (f"Accuracy across trials : {inference_stats_dict[ele]['acc_list']}")
  #print (f"Mean of errors : {np.mean(inference_stats_dict[ele]['cce_list'])}")
  #print (f"Variance of errors: {np.var(inference_stats_dict[ele]['cce_list'])}")

  print()

## Model: default ###
Cross entropy loss across trials : [0.0748375244140625, 0.05811812744140625, 0.08847588500976562, 0.029572921752929686, 0.061538922119140624, 0.08598414916992188, 0.07229287719726563, 0.07354020385742188, 0.08425131225585937, 0.07390631713867188]
Accuracy across trials : [0.5342, 0.6372, 0.4499, 0.8153, 0.616, 0.465, 0.5498, 0.5418, 0.4765, 0.5408]

## Model: L2 ###
Cross entropy loss across trials : [0.05077794799804688, 0.056246685791015626, 0.09849248657226563, 0.05702962646484375, 0.06205078125, 0.09080116577148438, 0.07283306884765625, 0.07203131103515625, 0.0712072998046875, 0.07413899536132812]
Accuracy across trials : [0.6847, 0.6514, 0.3908, 0.647, 0.6157, 0.4369, 0.5493, 0.5535, 0.5597, 0.5414]

## Model: dropout ###
Cross entropy loss across trials : [0.026508352661132814, 0.05739667358398438, 0.05832355346679687, 0.039481912231445315, 0.05254434814453125, 0.06762960205078125, 0.03944203796386719, 0.03917510681152344, 0.06364664306640624, 0.036692956542

In [67]:
import pandas as pd
df = pd.DataFrame()
df["seed"] = random_seeds
df["default_cce"] = inference_stats_dict["default"]["cce_list"]
df["L2_cce"] = inference_stats_dict["L2"]["cce_list"]
df["dropout_cce"] = inference_stats_dict["dropout"]["cce_list"]

df["default_acc"] = inference_stats_dict["default"]["acc_list"]
df["L2_acc"] = inference_stats_dict["L2"]["acc_list"]
df["dropout_acc"] = inference_stats_dict["dropout"]["acc_list"]

In [68]:
df

,seed,default_cce,L2_cce,dropout_cce,default_acc,L2_acc,dropout_acc
0,3058,0.074838,0.050778,0.026508,0.5342,0.6847,0.8314
1,3717,0.058118,0.056247,0.057397,0.6372,0.6514,0.6419
2,2781,0.088476,0.098492,0.058324,0.4499,0.3908,0.6363
3,2563,0.029573,0.057030,0.039482,0.8153,0.6470,0.7524
4,2120,0.061539,0.062051,0.052544,0.6160,0.6157,0.6714
5,2717,0.085984,0.090801,0.067630,0.4650,0.4369,0.5779
6,3823,0.072293,0.072833,0.039442,0.5498,0.5493,0.7522
7,2690,0.073540,0.072031,0.039175,0.5418,0.5535,0.7548
8,3371,0.084251,0.071207,0.063647,0.4765,0.5597,0.6031
9,3155,0.073906,0.074139,0.036693,0.5408,0.5414,0.7694


In [80]:
for ele in inference_stats_dict:
  print (f"## Model: {ele} ###")
  print (f"Mean of cross entropy : {np.mean(inference_stats_dict[ele]['cce_list'])}")
  print (f"Variance of cross entropy : {np.var(inference_stats_dict[ele]['cce_list'])}")
  print (f"Mean of accuracies : {np.mean(inference_stats_dict[ele]['acc_list'])}")
  print (f"Variance of accuracies : {np.var(inference_stats_dict[ele]['acc_list'])}")
  print ()

## Model: default ###
Mean of cross entropy : 0.07025182403564453
Variance of cross entropy : 0.00027028857743828755
Mean of accuracies : 0.56265
Variance of accuracies : 0.0103792525

## Model: L2 ###
Mean of cross entropy : 0.07056093688964844
Variance of cross entropy : 0.00020621667488921028
Mean of accuracies : 0.5630400000000001
Variance of accuracies : 0.0078768164

## Model: dropout ###
Mean of cross entropy : 0.04808411865234376
Variance of cross entropy : 0.00016590076974374424
Mean of accuracies : 0.6990799999999999
Variance of accuracies : 0.006309677600000001

